# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [1]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.functions import countDistinct

import pyspark.sql.functions as f

from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import FloatType

from pyspark.ml.feature import VectorAssembler, Normalizer, StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import re

import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Magic
%matplotlib inline

In [2]:
# create a Spark session
spark = SparkSession \
    .builder \
    .appName("Wrangling Data") \
    .getOrCreate()


In [3]:
spark.version

'2.4.3'

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [4]:
path = 'mini_sparkify_event_data.json'

user_log = spark.read.json(path)

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [5]:
#Look at the schema
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [6]:
#Look at some rows using pandas for more readable output
user_log.toPandas().head(10)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Martha Tilston,Logged In,Colin,M,50,Freeman,277.89016,paid,"Bakersfield, CA",PUT,NextSong,1.538173e+12,29,Rockpools,200,1538352117000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30
1,Five Iron Frenzy,Logged In,Micah,M,79,Long,236.09424,free,"Boston-Cambridge-Newton, MA-NH",PUT,NextSong,1.538332e+12,8,Canada,200,1538352180000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",9
2,Adam Lambert,Logged In,Colin,M,51,Freeman,282.82730,paid,"Bakersfield, CA",PUT,NextSong,1.538173e+12,29,Time For Miracles,200,1538352394000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30
3,Enigma,Logged In,Micah,M,80,Long,262.71302,free,"Boston-Cambridge-Newton, MA-NH",PUT,NextSong,1.538332e+12,8,Knocking On Forbidden Doors,200,1538352416000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",9
4,Daft Punk,Logged In,Colin,M,52,Freeman,223.60771,paid,"Bakersfield, CA",PUT,NextSong,1.538173e+12,29,Harder Better Faster Stronger,200,1538352676000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30
5,The All-American Rejects,Logged In,Micah,M,81,Long,208.29995,free,"Boston-Cambridge-Newton, MA-NH",PUT,NextSong,1.538332e+12,8,Don't Leave Me,200,1538352678000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",9
6,The Velvet Underground / Nico,Logged In,Micah,M,82,Long,260.46649,free,"Boston-Cambridge-Newton, MA-NH",PUT,NextSong,1.538332e+12,8,Run Run Run,200,1538352886000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",9
7,Starflyer 59,Logged In,Colin,M,53,Freeman,185.44281,paid,"Bakersfield, CA",PUT,NextSong,1.538173e+12,29,Passengers (Old Album Version),200,1538352899000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30
8,None,Logged In,Colin,M,54,Freeman,NaN,paid,"Bakersfield, CA",PUT,Add to Playlist,1.538173e+12,29,None,200,1538352905000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30
9,Frumpies,Logged In,Colin,M,55,Freeman,134.47791,paid,"Bakersfield, CA",PUT,NextSong,1.538173e+12,29,Fuck Kitty,200,1538353084000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30


In [7]:
#How many rows do we have?
user_log.count()

286500

In [ ]:
#Find out number of unique values per column and their datatypes
columns = user_log.schema.fieldNames()
datatypes = pd.DataFrame(user_log.dtypes, columns = ['column', 'dtype'])

for col in columns:
    nunique = user_log.select(col).dropDuplicates().count()
    dtype = datatypes[datatypes['column'] == col]['dtype'].values[0]
    output = 'Column: {} ({}) has {} unique values'.format(col, dtype, nunique)
    print(output)

Column: artist (string) has 17656 unique values
Column: auth (string) has 4 unique values
Column: firstName (string) has 190 unique values
Column: gender (string) has 3 unique values
Column: itemInSession (bigint) has 1322 unique values
Column: lastName (string) has 174 unique values
Column: length (double) has 14866 unique values
Column: level (string) has 2 unique values
Column: location (string) has 115 unique values
Column: method (string) has 2 unique values
Column: page (string) has 22 unique values


In [ ]:
#Look at the top artists
user_log.groupBy("artist").count().withColumnRenamed("count","distinct_name").sort(desc("count")).toPandas().head(20)

In [ ]:
#Look at the top auth
user_log.groupBy("auth").count().withColumnRenamed("count","distinct_name").sort(desc("count")).toPandas().head(20)

In [ ]:
#Look at the top gender
user_log.groupBy("gender").count().withColumnRenamed("count","distinct_name").sort(desc("count")).toPandas().head(20)

In [ ]:
##Look at the top  itemInSession
user_log.groupBy("itemInSession").count().withColumnRenamed("count","distinct_name").sort(desc("count")).toPandas()

In [ ]:
##Look at the top  level
user_log.groupBy("level").count().withColumnRenamed("count","distinct_name").sort(desc("count")).toPandas()

In [ ]:
##Look at the top  location
user_log.groupBy("location").count().withColumnRenamed("count","distinct_name").sort(desc("count")).toPandas()

In [ ]:
##Look at the top method
user_log.groupBy("method").count().withColumnRenamed("count","distinct_name").sort(desc("count")).toPandas()

In [ ]:
##Look at the top pages
user_log.groupBy("page").count().withColumnRenamed("count","distinct_name").sort(desc("count")).toPandas()

In [ ]:
##Look at the top registration
user_log.groupBy("registration").count().withColumnRenamed("count","distinct_name").sort(desc("count")).toPandas().head()

In [ ]:
##Look at the top status
user_log.groupBy("status").count().withColumnRenamed("count","distinct_name").sort(desc("count")).toPandas()

In [ ]:
##Look at the top userAgent
user_log.groupBy("userAgent").count().withColumnRenamed("count","distinct_name").sort(desc("count")).toPandas()

In [ ]:
##Look at the top userId
user_log.groupBy("userId").count().sort(desc("count")).toPandas().head(10)

In [ ]:
#We appear to have several thousand rows with missinng userId let's look at those pages
missing_userIds = user_log.select(["page", "level"]).where(user_log.userId == "")
missing_userIds.groupby("page").count().sort(desc("count")).toPandas().head(10)

The page names of the rows with missing Ids suggest this is probably activity prior to the user logging in.  We don't have any other kind of secondary ID, so we cannot attempt to associate these rows with an actual user.  The userAgent field isn't specific enough.  Because of this we must simply drop these rows.

In [ ]:
#create user_log_valid - removing rows without userId
user_log_valid = user_log.filter(user_log["userId"] != "")

In [ ]:
#We are interested in identifying when users cancel their account.  Create a function to flag this
flag_cancellation_event = udf(lambda x: 1 if x == "Cancellation Confirmation" else 0, IntegerType())

In [ ]:
#Use function to flag cancellation rows
user_log_valid = user_log_valid.withColumn("cancelled", flag_cancellation_event("page"))

In [ ]:
user_log_valid.toPandas().head(20)

In [ ]:
#Create grouped by dataframe
users_grouped = user_log_valid.groupby("userId").sum("cancelled").sort(desc("sum(cancelled)"))#.toPandas().head(10)
#Check we have removed the missing userId
#users_grouped.count().sort(desc("count")).toPandas().head(10)
users_grouped.show()

In [ ]:
users_grouped = user_log_valid.groupby("userId")\
                    .agg({"cancelled" : "sum",
                         "page" : "count",
                         "gender" : 'approx_count_distinct'})\
                    .withColumnRenamed("sum(cancelled)", "money")
users_grouped.show()

In [ ]:
#Create list of all the distinct page names
pages_list = list(user_log_valid.select("page").dropDuplicates().toPandas()["page"])
pages_list

In [ ]:
#Prepend "_event" to each page name for use as columns
pages_columns_renamed = ["event_" + x.replace(" ", "_") for x in pages_list]
pages_columns_renamed

In [ ]:
#Create dataframe of userId pivoted by page, to create a column of page views for each page 
user_page_events = user_log_valid.groupby("userId").pivot("page", pages_list).count()
#Replace NaN values with 0
user_page_events = user_page_events.fillna(0)
#Rename the columns, so each is prepended with "_event"
user_page_events = user_page_events.toDF(* ["userId"] + pages_columns_renamed)

In [ ]:
user_page_events.toPandas().head(10)

In [ ]:
user_page_events.toPandas().groupby("event_Cancellation_Confirmation").mean()

In [ ]:
user_log_valid.sort(['userId','sessionId']).toPandas().head(100)

In [ ]:
user_log.select(["userId", "firstname", "page", "song"]).where(user_log.userId == "1046").collect()

In [ ]:
user_log.groupby(["page"]).count().collect()

In [ ]:
#Pages
user_log.groupBy("page").count().withColumnRenamed("count","distinct_name").sort(desc("count")).toPandas()

In [ ]:
'''
'''
def check_columns_map_one_to_one(df, col_to_group_by, col_to_check_distincts):
    count_of_group_by_with_more_than_one = df.groupby([col_to_group_by]).\
        agg(countDistinct(col_to_check_distincts).\
        alias('unique_count')).\
        filter("unique_count > 1").count()
    return (count_of_group_by_with_more_than_one == 0)

In [ ]:
#There are several columns which sound like they might map 1-1 with a userId. 
#Check each column, if it does map 1-1 it is a candidate for being used as a feature
cols_to_check_are_unique_per_userId = ['gender', 'level', 'userAgent', 'status', 'registration', 'method', 'auth', 'location']

for col in cols_to_check_are_unique_per_userId:
    unique_check = check_columns_map_one_to_one(user_log_valid, 'userId', col)
    print('Every userId has a unique value for {} : {}'.format(col, unique_check))
    
#gender, userAgent, registraion and location all map 1-1 with a userId so are all candidate features

In [ ]:
gender_lookup = user_log_valid.groupBy("userId", "gender").count().select("userId", "gender")

is_female = udf(lambda x: 1 if x == "F" else 0, IntegerType())

gender_lookup = gender_lookup.withColumn("is_female", is_female("gender")).select("userId", "is_female")
gender_lookup.toPandas().head(200)

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [ ]:
'''
INPUT:
df

OUPUT:
df
'''
def engineer_features(df):
    
    #UDF for getting date string from timestamp
    get_date = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).strftime('%Y-%m-%d'))
    days_diff = udf(lambda x, y: (datetime.datetime.fromtimestamp(x / 1000.0)- datetime.datetime.fromtimestamp(y / 1000.0)).days)
    
    
    #Create date column from timestamp "YYYY-MM-DD" format
    df = df.withColumn("date", get_date(df.ts))
    
    
    #Create list of all the distinct page names
    pages_list = list(df.select("page").dropDuplicates().toPandas()["page"])
    #Prepend "_event" to each page name for use as columns
    pages_columns_renamed = ["event_" + x.replace(" ", "_") for x in pages_list]
    
    #Create dataframe of userId pivoted by page, to create a column of page views for each page 
    user_page_events = df.groupby("userId").pivot("page", pages_list).count()
    #Replace NaN values with 0
    user_page_events = user_page_events.fillna(0)
    #Rename the columns, so each is prepended with "_event"
    user_page_events = user_page_events.toDF(* ["userId"] + pages_columns_renamed)
    
    #The cancellation confirmation event is what we are trying to predict, we rename this "label"
    user_page_events= user_page_events.withColumnRenamed("event_Cancellation_Confirmation","label")
    
    gender_lookup = df.groupBy("userId", "gender").count().select("userId", "gender")
    
   
    #create UDF to map gender (F|M) to 1|0
    is_female = udf(lambda x: 1 if x == "F" else 0, IntegerType())
    #Map gender as 1|o in is_female column, drop gender column
    gender_lookup = gender_lookup.withColumn("is_female", is_female("gender")).select("userId", "is_female")
    
    #Join our userId feature tables together
    df_features = gender_lookup.join(user_page_events, on="userId", how="inner")
    
    #Create session count and 404 errors
    count_condition = lambda cond: f.sum(f.when(cond, 1).otherwise(0))
    df_user_features = df.groupBy("userId").agg(
        f.first('registration').alias('registration_ts'),
        countDistinct('sessionId').alias('sessions'),
        countDistinct('date').alias('unique_dates'),
        count_condition(f.col('status') == '404').alias('404_errors')
    )
    
    #Join session count and 404 errors
    df_features = df_features.join(df_user_features, on="userId", how="inner")
    
    
    #Group by userId and sessionId to be able to calculate average hits per session
    df_user_sessions = df.groupBy("userId", "sessionId").agg(
        countDistinct('itemInSession').alias('hitsInSession')
    )
    
    #Calculate average hits per session
    df_hits_per_session = df_user_sessions.groupBy("userId").agg(
        f.avg("hitsInSession").alias("hitsInSession_mean")
    )
    
    #Join hits_per_session to other features
    df_features = df_features.join(df_hits_per_session, on="userId", how="inner")
    
    #Get earliest timestamp in whole data set
    min_timestamp = df.agg({'ts': 'min'}).head()[0]
    
    #Convert registration timestamp to a date string "YYYY-MM-DD"
    df_features = df_features.withColumn("registration_date", get_date(df_features.registration_ts)).\
        withColumn("diff", f.round((f.lit(min_timestamp) - df_features.registration_ts)/1000/60/60/24))
    
    return df_features


def engineer_features2(df):
    
    count_condition = lambda cond: f.sum(f.when(cond, 1).otherwise(0))
    
    df_session_and_404 = df.groupBy("userId").agg(
        countDistinct('sessionId').alias('sessions'),
        count_condition(f.col('status') == '404').alias('404_errors')
    )
    
    df_user_sessions = df.groupBy("userId", "sessionId").agg(
        countDistinct('itemInSession').alias('hitsInSession')
    )
    
    df_hits_per_session = df_user_sessions.groupBy("userId").agg(
        f.avg("hitsInSession").alias("hitsInSession_mean")
    )
            
    
    return df_hits_per_session

In [ ]:
user_log_valid.groupBy("userId").agg({'registration' : 'first'}).head(5)

In [ ]:
averageCount = (user_log_valid
                .groupBy().mean()).head()[0]

In [ ]:
datetime.timedelta

In [ ]:
min_date = 1538348400000

get_hour = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0). hour)
get_date = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).strftime('%Y-%m-%d'))
days_diff = udf(lambda x, y: (datetime.datetime.fromtimestamp(x / 1000.0)- datetime.datetime.fromtimestamp(y / 1000.0).days))

                
user_log_time = user_log_valid. \
    withColumn("date", get_date(user_log_valid.ts)). \
    withColumn("date_registration", get_date(user_log_valid.registration)).\
    withColumn("diff", f.round((f.lit(min_date) - user_log_valid.registration)/1000/60/60/24))
    #withColumn("days_since_reg", days_diff(f.lit(min_date), user_log_valid.registration)).collect()



#newdf = df.select(to_date(‘date_from’, ‘yyyy-MM-dd HH:mm:ss’).alias(‘from_date’), to_date(‘date_to’, ‘yyyy-MM-dd HH:mm:ss’).alias(‘to_date’), )
#newdf.withColumn(“diff_days”, F.datediff(“to_date”, “from_date”)).show()


In [ ]:
#user_log_time.toPandas().head(10)

In [ ]:
#datetime.datetime.fromtimestamp(1538173362000 / 1000).strftime('%Y-%m-%d')

In [ ]:
#user_log_time.groupby('date').count().toPandas()

In [ ]:
df_features = engineer_features(user_log_valid)

In [ ]:
df_features.toPandas().head(30)

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [ ]:
numeric_feature_columns = df_features.columns
numeric_feature_columns.remove('userId')
numeric_feature_columns.remove('label')
numeric_feature_columns.remove('registration_ts')
numeric_feature_columns.remove('registration_date')
numeric_feature_columns

In [ ]:
#Assemble the Vector of Numeric features
assembler = VectorAssembler(inputCols=numeric_feature_columns, outputCol="NumFeatures")
df = assembler.transform(df_features)
#df.head() 

#Normalize the Vectors
scaler = Normalizer(inputCol="NumFeatures", outputCol="ScaledNumFeatures")
df =  scaler.transform(df)
df.head() 

In [ ]:
data = df.select("label","ScaledNumFeatures").withColumnRenamed('ScaledNumFeatures','features')
#data.toPandas().head()
data.head()

In [ ]:
#Model Training

lr = LogisticRegression(maxIter=5, regParam=0.0)
model = lr.fit(data)

In [ ]:
model.coefficientMatrix

In [ ]:
model.summary.accuracy

In [ ]:
#3 way split splits
#train, rest = df.randomSplit([0.6, 0.4], seed=40)
#test, validation = rest.randomSplit([0.5, 0.5], seed =40)
#train is now 60%, test is 20% and validation is 20%

In [ ]:
#2 way split
train, test = df_features.drop('userId').randomSplit([0.8, 0.2], seed=40)

In [ ]:
#pipeline

assembler = VectorAssembler(inputCols=numeric_feature_columns, outputCol="NumFeatures")
scaler = Normalizer(inputCol="NumFeatures", outputCol="features")
lr = LogisticRegression(maxIter=5, regParam=0.0)

pipeline = Pipeline(stages=[assembler, scaler, lr])

pipeline_model = pipeline.fit(train)



In [ ]:
predictions = pipeline_model.transform(test)

In [ ]:
#important: need to cast to float type, and order by prediction, else it won't work
predictions_and_labels = predictions.select(['prediction','label']).withColumn('label', f.col('label').cast(FloatType())).orderBy('prediction')

#select only prediction and label columns
predictions_and_labels = predictions_and_labels.select(['prediction','label'])

metrics = MulticlassMetrics(predictions_and_labels.rdd.map(tuple))

print(metrics.confusionMatrix().toArray())

In [ ]:
len(y_true)

In [ ]:
y_true = predictions.select(['label']).collect()
y_pred = predictions.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))

In [ ]:
predictions.toPandas().head()

In [ ]:
predictions.filter(predictions.label == predictions.prediction).count()

In [ ]:
#crossval = CrossValidator(estimator = pipeline)

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.